# Music Popularity Analysis

<hr style="border:2px solid black"> </hr>

## Notebook 04 - last.fm API

---

### Import libraries

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import requests
import sys

In [3]:
def num_uniques(ser):
    try:
        return len(ser.unique())
    except:
        return "Not unique check-able"

In [4]:
def summarize_df(df):
    print("======DATA SUMMARY======")
    print("{} rows by {} columns".format(df.shape[0], df.shape[1]))
    print("\n======COLUMNS======")
    print(df.dtypes)
    print("\n======PREVIEW======")
    display(df.head())
    print("\n======NUMERICAL COL SUMMARY======")
    print(df.describe())
    print("\n")
    for col in df.columns:
        print("{}: {} unique values".format(col, num_uniques(df[col])))

---
### Read in data

In [5]:
api_key = '6b198bd00d4f795ff68eadf6054ebe60'

In [15]:
rel_name_artist = pd.read_csv('rel_name_artist.csv')[['release_gid', 'track_name', 'artist_name']]
rel_name_artist.head()

,release_gid,track_name,artist_name
0,56d792d8-2482-43d8-a552-8ab13566a106,Am I Dreaming,Dionne Warwick
1,56d792d8-2482-43d8-a552-8ab13566a106,Tears Ago,Dionne Warwick
2,56d792d8-2482-43d8-a552-8ab13566a106,What Color Is Love,Dionne Warwick
3,56d792d8-2482-43d8-a552-8ab13566a106,How To Keep The Music Playing,Dionne Warwick
4,56d792d8-2482-43d8-a552-8ab13566a106,Déjà Vu,Dionne Warwick


In [16]:
spot_isrc_uri = pd.read_csv('isrc_uri.csv')[['isrc', 'sp_uri']]
spot_isrc_uri.head()

,isrc,sp_uri
0,DEA622100038,3BDcuuLSAUCp3V5gtXmU3M
1,QZES92183707,3HWIktiW8cv1KcHLc7iYIW
2,USUM72001306,75nKBP8jQu681pTNCtrEnn
3,USEE10183941,6wGMLoEqD2k3RBXFHD0yUp
4,RUA582002334,15J6jZN7MM4gG617FyohOQ


In [17]:
full_set = pd.read_parquet('mb_db_songs_sample.parquet')
full_set.head()

,track_id,track_gid,recording_id,medium_id,track_name,recording_gid,isrc,release_id,release_gid,release_name,artist_credit_id,barcode,artist_credit_name,date_year
5138944,30694977,f315f0e6-8b85-4af2-b18b-6015e552d422,3485200,2870106,What the World Needs Now Is Love (Dionne Sings...,e852fa84-c2be-4104-90db-8ef118566f28,USCMG9800120,2643114,56d792d8-2482-43d8-a552-8ab13566a106,She’s Back,16172,634164608521,Dionne Warwick,2019
6379854,28274156,fed2cf3e-2053-4282-ae8a-aca89ded4027,24794010,2626676,なんてこった,b51f01d5-fce5-44ae-ad0b-b32ac9a54e0b,JPVI01900906,2424086,5c8fd7dd-e054-44b3-b7d4-1b422b8c344a,TVアニメ「私に天使が舞い降りた！」サウンドコレクション,1821450,4580325327977,伊賀拓郎,2019
7784570,36445759,52519c42-73c9-468b-b44a-6ff5ad88879b,30688971,3506956,"La nemica d’amore fatta amante, serenata à 3: ...",5cc269f2-b522-4b44-b568-e3d3e3d3e002,FR1P70201730,3218908,3fd04337-6682-4cf8-8e7c-d5330c7c933c,La nemica d’amore fatta amante,3124554,3760014196119,"Bononcini; Adriana Fernández, Martín Oro, Furi...",2021
7674635,33863813,6efedc41-368f-461b-b83c-0fcd2b1b2165,28792342,3214362,Chill Your Eyes,a04503c7-72ff-4ffc-9538-9e79a95564ec,DEW872001575,2953727,bcb6d88c-288a-48e0-a367-385c5fbbc879,"Coffee Bar Lounge, Vol. 20",1,4251794433321,Various Artists,2020
6516875,29738294,e467ccc6-dcc1-461c-82e4-69a10dcc7f4b,25854890,2774306,Ouverture,7133e0a5-6c17-4478-92a5-124baf17e4b6,CAA4H1903001,2557577,56cd6377-bc3e-4f26-8300-98bc1f879263,De temps et de vents,1599253,3616402095765,Bodh’aktan,2019


In [18]:
release_gids = full_set.merge(spot_isrc_uri, on='isrc')['release_gid'].unique()

In [27]:
rel_name_artist = rel_name_artist.merge(pd.DataFrame(pd.DataFrame(release_gids, columns=['release_gid'])), on='release_gid')
rel_name_artist = rel_name_artist.values.tolist()

AttributeError: 'list' object has no attribute 'merge'

In [29]:
pd.DataFrame(rel_name_artist, columns=['release_gid', 'track_name', 'artist_name']).to_csv('rel_name_artist_slim.csv')


In [49]:
# Initialize from file - TBD

In [30]:
# Initialize from start
k = 0
gid_name_art_count = []
fails=[]

In [51]:
k = 78700

In [63]:
for k in range(k, len(rel_name_artist)):
    gid = rel_name_artist[k][0]
    name = rel_name_artist[k][1]
    artist = rel_name_artist[k][2]
#     print(artist, name)

#     name_enc = urllib.parse.quote(name.encode('utf8'))
#     artist_enc = urllib.parse.quote(artist.encode('utf8'))
    
    retries = 1
    success = False

    while not success:
        try:
            r = requests.get('https://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={}&artist={}&track={}&format=json'.format(api_key, artist, name))
            success = True
        except Exception as e:
            wait = retries * 30;
            print(f'Error! Waiting {wait} secs and re-trying...')
            sys.stdout.flush()
            time.sleep(wait)
            retries += 1
    try:
        if 'track' in r.json().keys():
            gid_name_art_count.append([gid, name, artist, r.json()['track']['playcount']])
        else:
            fails.append(k)
    except:
        fails.append(k)
    k += 1
    if k % 100 == 0:
        print('{} songs looked up, {} found'.format(k, len(gid_name_art_count)))

291500 songs looked up, 232327 found
291600 songs looked up, 232426 found
291700 songs looked up, 232525 found
291800 songs looked up, 232614 found
291900 songs looked up, 232713 found
292000 songs looked up, 232796 found
292100 songs looked up, 232880 found
292200 songs looked up, 232974 found
292300 songs looked up, 233043 found
292400 songs looked up, 233142 found
292500 songs looked up, 233241 found
292600 songs looked up, 233341 found
292700 songs looked up, 233426 found
292800 songs looked up, 233520 found
292900 songs looked up, 233597 found
293000 songs looked up, 233689 found
293100 songs looked up, 233761 found
293200 songs looked up, 233850 found
293300 songs looked up, 233937 found
293400 songs looked up, 234028 found
293500 songs looked up, 234087 found
293600 songs looked up, 234173 found
293700 songs looked up, 234272 found
293800 songs looked up, 234371 found
293900 songs looked up, 234470 found
294000 songs looked up, 234556 found
294100 songs looked up, 234646 found
2

In [64]:
# Save out to file
pd.DataFrame(gid_name_art_count, columns=['gid', 'track_name', 'artist_name', 'play_count']).to_csv('gid_name_art_count.csv')

In [65]:
# Save out to file
pd.DataFrame(fails, columns=['fail_id']).to_csv('gid_name_art_count_fails.csv')